In [28]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sat Jul  4 03:25:46 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [29]:
from google.colab import drive
drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [30]:
import os
os.chdir('/drive/My Drive')

In [31]:
#pip install tensorflow==1.14.0

In [32]:
import warnings

In [33]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.resnet import ResNet50
from keras.applications.resnet import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob


import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [34]:
IMAGE_SIZE = [224, 224]

In [35]:
resnet = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [36]:
# don't train existing weights
for layer in resnet.layers:
  layer.trainable = False

In [37]:
x = Flatten()(resnet.output)
prediction = Dense(2, activation='softmax')(x)
model = Model(inputs=resnet.input, outputs=prediction)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [38]:
from keras import optimizers

model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

In [39]:
# Data Augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [40]:
# Data Augmentation
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [41]:
# Make sure you provide the same target size as initialied for the image size
train_set = train_datagen.flow_from_directory('/drive/My Drive/IMAGE_RECOGNITION/DOG vs CAT/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 8005 images belonging to 2 classes.


In [42]:
test_set = test_datagen.flow_from_directory('/drive/My Drive/IMAGE_RECOGNITION/DOG vs CAT/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 2023 images belonging to 2 classes.


In [43]:
from datetime import datetime
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau

#lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

#num_epochs = 1000
#num_batch_size = 32

checkpoint = ModelCheckpoint(filepath='/drive/My Drive/IMAGE_RECOGNITION/DOG vs CAT/my_model.h5', 
                               verbose=1, save_best_only=True)

callbacks = [checkpoint, lr_reducer]

start = datetime.now()

model.fit_generator(
  train_set,
  validation_data=test_set,
  steps_per_epoch=200,
  epochs=30,
  validation_steps=15,
    callbacks=callbacks ,verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/30
200/200 [==============================] - 1271s 6s/step - loss: 1.0697 - accuracy: 0.9085 - val_loss: 0.9612 - val_accuracy: 0.9208

Epoch 00001: val_loss improved from inf to 0.96119, saving model to /drive/My Drive/IMAGE_RECOGNITION/DOG vs CAT/my_model.h5
Epoch 2/30
200/200 [==============================] - 777s 4s/step - loss: 0.9419 - accuracy: 0.9303 - val_loss: 0.4922 - val_accuracy: 0.9292

Epoch 00002: val_loss improved from 0.96119 to 0.49224, saving model to /drive/My Drive/IMAGE_RECOGNITION/DOG vs CAT/my_model.h5
Epoch 3/30
200/200 [==============================] - 719s 4s/step - loss: 0.8710 - accuracy: 0.9368 - val_loss: 0.4844 - val_accuracy: 0.9542

Epoch 00003: val_loss improved from 0.49224 to 0.48443, saving model to /drive/My Drive/IMAGE_RECOGNITION/DOG vs CAT/my_model.h5
Epoch 4/30
200/200 [==============================] - 723s 4s/step - loss: 0.6986 - accuracy: 0.9483 - val_loss: 0.5374 - val_accuracy: 0.9625

Epoch 00004: val_loss did not improve fr

In [68]:
model.evaluate(train_set)

251/251 [==============================] - 703s 3s/step - loss: 0.3918 - acc: 0.9725


[0.3918233374176782, 0.9725172]

In [69]:
model.evaluate(test_set)

64/64 [==============================] - 177s 3s/step - loss: 0.3807 - acc: 0.9738


[0.38074484577219914, 0.9738013]

In [70]:

from tensorflow import keras
model = keras.models.load_model('/drive/My Drive/IMAGE_RECOGNITION/DOG vs CAT/my_model.h5')

In [84]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('/drive/My Drive/IMAGE_RECOGNITION/DOG vs CAT/test/cats/2.jpg', target_size = (224,224))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)


In [85]:
result

array([[1., 0.]], dtype=float32)

In [87]:
train_set.class_indices

{'cats': 0, 'dogs': 1}

In [88]:
train_set.class_indices
if result[0][0]:
    prediction = 'cat'
    print(prediction)
else:
    prediction = 'dog'
    print(prediction)

cat
